# DATA SYNOPSIS

In [ ]:
#importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import random
import string
import warnings
warnings.filterwarnings("ignore")
import datetime
from datetime import timedelta
import time

In [ ]:
#importing data
apprecord_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRrUs2YukMSuRAufitU9hKvEm76XDbf-8l4zWGnnZZr1nunuUL_V6tPga1gZikXjnADpBlNJAzyU9s_/pub?output=csv")
credrecord_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQ-olGb3tNztV0BdJj7xR3IPUOoLsRmg4zAQN72sa3ixryOOrWWEJKi7MfOHI_tYroLIvkFSbicxF2O/pub?output=csv")

In [ ]:
#viewing data
display(apprecord_df.head(5),apprecord_df.shape)
display(credrecord_df.head(5),credrecord_df.shape)

In [ ]:
time1 = time.time()
data = pd.merge(apprecord_df,credrecord_df,on="ID")
df_all = data
display(df_all,df_all.shape)

In [ ]:
display(apprecord_df[apprecord_df["ID"]==5008804])
display(credrecord_df[credrecord_df["ID"]==5008804])

In [ ]:
for column in df_all.columns:
  display(df_all[column].value_counts())

In [ ]:
display(df_all[df_all["DAYS_EMPLOYED"]== 365243]["NAME_INCOME_TYPE"].value_counts())

In [ ]:
df_all.describe(include = "all")

In [ ]:
df_all.info()

In [ ]:
df_all.corr()

#DATA CLEANING

In [ ]:
#STEPS
#check missing values
#handling missing values
#cleaning wrong data
#check duplicate values
#drop unneccesary columns


In [ ]:
# checking missing values
df_all.isna()

In [ ]:
df_all.isna().any()

In [ ]:
df_all.isna().sum()

In [ ]:
#handling missing values
#here we can see the occupation type has missing values
df_all["OCCUPATION_TYPE"].value_counts()

In [ ]:
#filling the missing values in the column OCCUPATIONAL_TYPE
df_all["OCCUPATION_TYPE"].fillna("Others",inplace = True)

In [ ]:
#checking again for null values
df_all.isna().any()

In [ ]:
#in DAYS_WORKED column we can find value which is 365243
"""for x in df_all.index:
    if df_all.loc[x,"DAYS_EMPLOYED" ]== 365243:
      df_all.loc[x,"DAYS_EMPLOYED"]= 36500
df_all["DAYS_EMPLOYED"]"""
#this is not not recommended as it is taking much time
#loc and iloc are useful for getting multiple items at a time
#at and iat are useful for getting single value at a time

In [ ]:
#cleaning wrong data
#in DAYS_WORKED column we can find value which is 365243
"""for x in df_all.index:
    if df_all.at[x,"DAYS_EMPLOYED" ]== 365243:
      df_all.at[x,"DAYS_EMPLOYED"]= 36500
df_all["DAYS_EMPLOYED"]"""

In [ ]:
#cleaning wrong data
#in DAYS_WORKED column we can find value which is 365243
df_all.loc[df_all['DAYS_EMPLOYED']== 365243,'DAYS_EMPLOYED']= 36500

In [ ]:
#checking for duplicated columns
df_all.duplicated().sum()

In [ ]:
#removing the unneccesary columns
#as per correlation we find FLAG_MOBIL is not correlated to any column
try:
  df_all.drop("FLAG_MOBIL",axis=1,inplace = True)
except:
  print("already removed or does not exist")

In [ ]:
df_all

#FEATURE ENGINEERING

In [ ]:
#steps
#creating temporal variables
#categorical variables : remove rare labels
#standardise the value of variables to the same range

In [ ]:
#creating temporal variables
df_all["CODE_GENDER"].value_counts()

In [ ]:
dumgendf =pd.get_dummies(df_all["CODE_GENDER"])

In [ ]:
#male = dumgendf["M"]
female = dumgendf["F"]
male = dumgendf.loc[:,"M":"M"]
display(male.value_counts()) #male value is 1 and female value is 0
male.columns = ["MALE_GENDER"]
male.columns

In [ ]:
car = pd.get_dummies(df_all["FLAG_OWN_CAR"],drop_first = True)
display(car.value_counts())
car.columns = ["OWN_CAR"]
car.columns

In [ ]:
OWN_LAND = pd.get_dummies(df_all["FLAG_OWN_REALTY"],drop_first = True)
display(OWN_LAND.value_counts())
OWN_LAND.columns = ["OWN_LAND"]
OWN_LAND.columns

In [ ]:
"""listofdum = [male, car, OWN_LAND]
try:
  for column in listofdum:
    df_all.insert(2,f"{column.columns[0]}",column)
    display(df_all)
except:
  display(df_all)"""

In [ ]:
df_all =pd.concat([df_all,male,car,OWN_LAND],axis =1)

In [ ]:
df_all

In [ ]:
#categorical variables : remove rare labels
# try:
df_all.drop("CODE_GENDER",axis=1, inplace = True)
df_all.drop("FLAG_OWN_CAR", axis = 1, inplace = True)
df_all.drop("FLAG_OWN_REALTY", axis =1, inplace = True)
#   print("function applied for first time")
# except:
#   print("function already applied")

In [ ]:
df_all

In [ ]:
def education(x):
  if x == "Secondary / secondary special":
    x=x.split(" / ")[0]
  return x

In [ ]:
df_all['NAME_EDUCATION_TYPE'] = df_all['NAME_EDUCATION_TYPE'].apply(education)
df_all['NAME_EDUCATION_TYPE'].value_counts()

In [ ]:
def marriage(x):
  if x == "Single / not married":
    x = x.split(" / ")[0]
  return x

In [ ]:
df_all["NAME_FAMILY_STATUS"] = df_all["NAME_FAMILY_STATUS"].apply(marriage)
df_all["NAME_FAMILY_STATUS"].value_counts()

In [ ]:
def house(x):
  if x == "House / apartment":
    x = x.split(" / ")[0]
  return x

In [ ]:
df_all["NAME_HOUSING_TYPE"] = df_all["NAME_HOUSING_TYPE"].apply(house)
df_all["NAME_HOUSING_TYPE"].value_counts()

In [ ]:
df_all.head(5)

In [ ]:
#converting DAYS_BIRTH and DAYS_EMPLOYED into normal date
def date_to_time_fun(totaldays):
  dttfun = (datetime.date.today()+timedelta(totaldays)).strftime("%d-%m-%Y")
  return dttfun
print(date_to_time_fun(-2333))

In [ ]:
# try:
df_all['DAYS_BIRTH'] = df_all["DAYS_BIRTH"].apply(date_to_time_fun)
df_all["DAYS_EMPLOYED"]= df_all["DAYS_EMPLOYED"].apply(date_to_time_fun)
# print("function applied for first time")
# except:
#   print("function already applied")


In [ ]:
display(df_all[["DAYS_EMPLOYED","DAYS_BIRTH"]])
df_all["DAYS_EMPLOYED"].value_counts()

In [ ]:
#getting number of years
def noofyearfun(date_of_time):
  today = datetime.date.today().year
  yod = datetime.datetime.strptime(date_of_time,"%d-%m-%Y").year
  noyears = today - yod
  return noyears
print(noofyearfun("07-08-2016"))


In [ ]:
# try:
df_all["DAYS_BIRTH"] = df_all["DAYS_BIRTH"].apply(noofyearfun)
# print("function applied first time")
# except:
#   print("function already applied")

In [ ]:
# try:
df_all["DAYS_EMPLOYED"] = df_all['DAYS_EMPLOYED'].apply(noofyearfun)
# print("function applied for first time")
# except:
#   print("function already applied")

In [ ]:
df_all = df_all.rename(columns = {"DAYS_BIRTH": "AGE", "DAYS_EMPLOYED" : "NO_OF_YEARS_WORKED"})

In [ ]:
with pd.option_context('display.max_columns', None,
                       'display.precision', 3,
                       ): # u can also insert 'display.max_rows', None
  display(df_all)

In [ ]:
df_all["STATUS"].value_counts()

In [ ]:
# MONTHS_BALANCE
# record month: The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on

# STATUS
# 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5:
df_all.loc[df_all['STATUS']=='C','STATUS']=6
df_all.loc[df_all['STATUS']=='X','STATUS']=7

In [ ]:
df_all["STATUS"] = df_all["STATUS"].astype(float)

In [ ]:
def solve(state):
  credit_score = ""
  if int(state) in (2,3,4,5):
    credit_score = 1  #risky
  else:
    credit_score = 0 #not risky
  return credit_score

In [ ]:
df_all["Credit"] = df_all["STATUS"].apply(solve)

In [ ]:
df_all["Credit"].value_counts()

In [ ]:
# try:
#   df_all.drop("STATUS",axis = 1,inplace = True)
#   print("function performed for first time")
# except:
#   print("function already performed")

In [ ]:
df_all

# DATA VISUALISATION

In [ ]:
df_all.corr()

In [ ]:
plt.figure(figsize = (30,8))
sns.heatmap(df_all.corr(),annot = True, cmap = "RdBu") #cmap = "RdYlGn")

In [ ]:
plt.figure(figsize = (20,10))
vc_pie  = df_all["NAME_FAMILY_STATUS"].value_counts()  #valuecounts for piechart    `1`
num = vc_pie.keys()
plt.pie(vc_pie,labels =vc_pie.keys(),explode = np.linspace(0,(1/len(num)),len(num)) )

"""From the above pie chart ,we conclude that majority of people are married so there expenses more than others.so, we can approve their credit card bcz they can use credit card regularly and in efficient manner for their expenses."""

In [ ]:
plt.figure(figsize = (20,10))
num =df_all["NAME_HOUSING_TYPE"].unique()
vc_pie  = df_all["NAME_HOUSING_TYPE"].value_counts()
plt.pie(vc_pie,labels =vc_pie.keys(),explode = np.linspace(0,(1/len(num)),len(num)) )

"""From this above pie chart , we conclude that most of the people lives in their own house or apartment.So There is a high chance that they are able to pay their bills on time ,so we can approve their credit card"""

In [ ]:
plt.figure(figsize = (15,10))
itforpie = df_all[df_all["Credit"] == 1]["NAME_INCOME_TYPE"].value_counts() #incometype for piechart
plt.pie(itforpie,labels = itforpie.keys(),explode = np.linspace(0,1/len(itforpie.keys()),len(itforpie.keys())))
"""here we can see person with income type 'student' have not skipped the loan """

In [ ]:
plt.figure(figsize = (15,10))
fsforpie = df_all[df_all["Credit"] == 1]["NAME_FAMILY_STATUS"].value_counts() #familystatusforpiechart
plt.pie(fsforpie,labels = fsforpie.keys(),explode = np.linspace(0,1/len(fsforpie.keys()),len(fsforpie.keys())))
"""here we can see person with family status 'Married' have skipped the loan """

In [ ]:
dfcopy = df_all.copy()
for feature in df_all.columns:
  if feature != "STATUS":
    plt.scatter(dfcopy[feature],dfcopy["STATUS"])
    plt.xlabel(feature)
    plt.ylabel("STATUS")
    plt.show()

In [ ]:
#requires more ram
sns.pairplot(df_all["CNT_CHILDREN	AMT_INCOME_TOTAL	AGE	NO_OF_YEARS_WORKED	FLAG_WORK_PHONE	FLAG_PHONE	FLAG_EMAIL	CNT_FAM_MEMBERS	MONTHS_BALANCE	MALE_GENDER	OWN_CAR	OWN_LAND	Credit".split()[:4]])

# FEATURE SCALING


In [ ]:
# let's make all data in one scale,for that we have to first change the categorical data to numerical data .
df_all.head(10)

In [ ]:
# here OCCUPATION_TYPE doesnot have any relation with credit
df_all.drop("OCCUPATION_TYPE",axis = 1, inplace = True)

In [ ]:
dfforbot1 = df_all.drop(["ID","FLAG_WORK_PHONE","FLAG_PHONE","FLAG_EMAIL","MONTHS_BALANCE","STATUS"],axis = 1).copy()

In [ ]:
dfforbot1.NAME_EDUCATION_TYPE.value_counts()

In [ ]:
#changing NAME_INCOME_TYPE	NAME_EDUCATION_TYPE	NAME_FAMILY_STATUS	NAME_HOUSING_TYPE columns into numerical data
def changetonum(lis):
  lempty = [ ]
  for i in range(1,len(lis)+1):
    lempty.append((lis[i-1],i))
  return dict(lempty)
a,b,c,d = "NAME_INCOME_TYPE	NAME_EDUCATION_TYPE	NAME_FAMILY_STATUS	NAME_HOUSING_TYPE".split()
lrp1 = changetonum(list(df_all[a].value_counts().keys()))
lrp2 = changetonum(list(df_all[b].value_counts().keys()))
lrp3 = changetonum(list(df_all[c].value_counts().keys()))
lrp4 = changetonum(list(df_all[d].value_counts().keys()))

In [ ]:
(lrp2)

In [ ]:
df_all.replace({a:lrp1},inplace=True)
df_all.replace({b:lrp2},inplace = True)
df_all.replace({c:lrp3},inplace = True)
df_all.replace({d:lrp4},inplace = True)
df_all

In [ ]:
#creating a copy for not modifying the data in original data frame
df_all1 = df_all.copy()
df_all.head(1)

In [ ]:
df_all1 = df_all1.drop(["ID","FLAG_WORK_PHONE","FLAG_PHONE","FLAG_EMAIL","MONTHS_BALANCE","STATUS"],axis =1)
df_all1

In [ ]:
df_all1.shape

In [ ]:
df_all1.drop(["MALE_GENDER"],axis = 1 , inplace =True)
df_all1 = df_all1.drop_duplicates()
# df_all1[df_all1["NO_OF_YEARS_WORKED"]==-100]=-100
df_all1.info()

In [ ]:
"""We can see that our all data in float or in int data types.Now we ready to apply FeatureScaling on our data."""

In [ ]:
x = df_all1.drop("Credit",axis =1) #independent feature
y = df_all1["Credit"]     #dependent feature or predicting value

In [ ]:
feature_scale = [feature for feature in df_all1.columns if feature != "Credit"]
#importing libraries
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()
scaler.fit(x)

In [ ]:
(scaler.transform(x))

In [ ]:
df_all1 = pd.concat([y.reset_index(drop = True),pd.DataFrame(scaler.transform(x),columns=feature_scale)],axis =1)
df_all1.head(5)

In [ ]:
df_all1.Credit.value_counts()

In [ ]:
""" from the above output,we conclude that data is imbalanced.so, we have to do either under sampling or over sampling.we are going to do over sampling bcz in under sampling .we will lost our almost data."""

# DEALING WITH UNBALANCED DATA SET

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
smtk = SMOTETomek(random_state=12)

In [ ]:
df_all1.shape

In [ ]:
y = y.astype(int)
x_resamp , y_resamp = smtk.fit_resample(x,y)

In [ ]:
x_resamp.shape,y_resamp.shape

In [ ]:
print(f"original data set value distribution : {list(y.value_counts())}")
print(f"resampled data set value distribution : {list(y_resamp.value_counts())}")

In [ ]:
#x_resamp.groupby(["MONTHS_BALANCE","ID"]).min().nsmallest(10,columns ="CNT_CHILDREN	AMT_INCOME_TOTAL	NAME_INCOME_TYPE	NAME_EDUCATION_TYPE	NAME_FAMILY_STATUS	NAME_HOUSING_TYPE	AGE	NO_OF_YEARS_WORKED	FLAG_WORK_PHONE	FLAG_PHONE	FLAG_EMAIL	CNT_FAM_MEMBERS	STATUS".split())

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
x_resamp1 = x_resamp
feature_scale1 = feature_scale

In [ ]:
mms = MinMaxScaler()
mms.fit(x_resamp1)

In [ ]:
x_scaled_resamp = mms.transform(x_resamp1)

In [ ]:
df_all1scaled = pd.concat([pd.DataFrame(x_scaled_resamp,columns = feature_scale1),y_resamp],axis =1)

In [ ]:
DF1111 = df_all1scaled[df_all1scaled["Credit"]==1 ]
DF1111

# MODEL DEPLOYMENT

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#pd.concat([x_resamp,y_resamp],axis = 1)[pd.concat([x_resamp,y_resamp],axis = 1)["Credit"] == 1]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled_resamp,y_resamp,train_size = 0.8,random_state = 567)

In [ ]:
x_train.shape,y_train.shape

In [ ]:
#importing libraries
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay,accuracy_score

In [ ]:
#importing libraries
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay,accuracy_score

In [ ]:
modellist = [LinearRegression,DecisionTreeClassifier,GaussianNB,KNeighborsClassifier,RandomForestClassifier]

In [ ]:
accuracylist = []
i=0
for model in modellist:
  model = model()
  model.fit(x_train,y_train)
  y_pred =model.predict(x_test)
  print("one iteration completed")
  ConfusionMatrixDisplay.from_predictions(y_test ,y_pred.astype(int))
  acc = accuracy_score(y_test,y_pred.astype(int))
  accuracylist.append(acc)
  i+=1

In [ ]:
"""accuracylist = []
def model_test(x_train,y_train,model):
  model = model()
  model.fit(x_train,y_train)
  y_pred =model.predict(x_test)
  ConfusionMatrixDisplay.from_predictions(y_test ,y_pred)
  acc = accuracy_score(y_test,y_pred)
  accuracylist.append(acc)
  return (y_pred)

ypredl = [model_test(x_train,y_train,i) for i in modellist ]
ypredl"""

In [ ]:
display(modellist)
accuracylist

In [ ]:
maxac = max(accuracylist)
acindex = accuracylist.index(maxac)
modelforpred = modellist[acindex]
modelforpred

# CHATBOT NORMAL

In [ ]:
#steps
#separating multifeature data and numerical data
#getting input for multifeature data
#getting numerical data
#arranging the input data as per the database

In [ ]:
dataforbot = dfforbot1.drop(["Credit","MALE_GENDER"],axis = 1)
dataforbot.head(5)

In [ ]:
#inputlist = "ID	CODE_GENDER	FLAG_OWN_CAR	FLAG_OWN_REALTY	CNT_CHILDREN	AMT_INCOME_TOTAL	NAME_INCOME_TYPE	NAME_EDUCATION_TYPE	NAME_FAMILY_STATUS	NAME_HOUSING_TYPE	DAYS_BIRTH	DAYS_EMPLOYED	FLAG_WORK_PHONE	FLAG_PHONE	FLAG_EMAIL	OCCUPATION_TYPE	CNT_FAM_MEMBERS	MONTHS_BALANCE".split()
inputlist  = list(dataforbot.columns)
inputlist

In [ ]:
dataforbot.describe(include = "all")

In [ ]:
numdatalist = dataforbot.describe().loc["std",:][dataforbot.describe().loc["std",:] >0.5]
numdatalist = numdatalist.keys() #it takes only if the std is greater than 0.5
numdatalist

In [ ]:
#numdatalist = ["ID","AMT_INCOME_TOTAL","CNT_CHILDREN","DAYS_BIRTH","AGE","DAYS_EMPLOYED", "NO_OF_YEARS_WORKED","CNT_FAM_MEMBERS","MONTHS_BALANCE"]

In [ ]:
"""inpdetlist = [[list(dataforbot[i].value_counts().keys()) , list(dataforbot[i].value_counts())] for i in inputlist if i not in numdatalist]
inpdetlist"""

In [ ]:
#working for non num data
nonnumlist = [column for column in inputlist if column not in numdatalist]
# indetlist1 = [list(enumerate(dataforbot[column].unique(),1)) for column in nonnumlist]
indetlist1 = [list(enumerate(dataforbot[column].value_counts().keys(),1)) for column in nonnumlist]
display(nonnumlist)
indetlist1

In [ ]:
#non mumerical data
def funtionnonnuminp(ind,col):
  i = 1
  while i !=0:
    try :
      print(f"Enter your {ind}  :  NOTE: IN FLAG IF 0 AND 1 ARE GIVEN, 0 MEANS NO AND 1 MEANS YES")
      input1 = int(input(f"SELECT THE NUMBER FROM THE LIST{col}"))
      if input1>len(col) or input1<1:
        raise ValueError("enter correct number")
      inpnonnumdata.append(input1)
      i=0
    except ValueError:
      print("enter correct number from list")

In [ ]:
num = len(nonnumlist)
inpnonnumdata = []
for j in range(num):
  nameofcol = nonnumlist[j]   #0
  valcountsofcol = indetlist1[j]#0
  funtionnonnuminp(nameofcol,valcountsofcol)

inpnonnumdata

In [ ]:
datainput1 = []
for i in range(len(inpnonnumdata)):
  datainput1.append((nonnumlist[i],indetlist1[i][inpnonnumdata[i]-1][1]))
print("entered details are : ")
datainput1

In [ ]:
#dealing with numerical inputs
#for arranging in order of the columns in data given
a = dataforbot
numcol = [col for col in a.columns if col in numdatalist]
numcol

In [ ]:
#getting the border values of the data for bot columns
minmaxlist = []
for col in numcol:
  if dataforbot[col].max() >0:
    minmaxlist.append(dataforbot[col].max())
  else:
    minmaxlist.append(dataforbot[col].min())
minmaxlist

In [ ]:
#getting numerical data
def funtionnuminp(ind,col):
  i = 1
  while i !=0:
    try :
      print(f"Enter your {ind}  : ")
      input1 = int(input(f"SELECT ONLY NUMBER FROM 0 to {col} : "))
      if col >0:
        if input1>col or input1<0:
          raise ValueError("enter correct number")
      elif col <0:
        if input1 <col or input1>0:
          raise ValueError("enter correct number")
      inpnumdata.append(input1)
      i=0
    except ValueError:
      print("enter correct number within range specified")

In [ ]:

inpnumdata = []
for j in range(len(numcol)):
  nameofcolumn = numcol[j]
  bordervalue = minmaxlist[j]
  funtionnuminp(nameofcolumn, bordervalue)
inpnumdata

In [ ]:
datainput2 = list(zip(numcol,inpnumdata))
datainput2

In [ ]:
#getting num variables that are initiated for the data
inpnonnumdata+ inpnumdata

In [ ]:
dataforinptomodel = datainput1 + datainput2
dataforinptomodel


In [ ]:
display(inpnonnumdata)
display(dfforbot1.head())
dfforbot1["OWN_CAR"].value_counts()

In [ ]:
xforinp = pd.DataFrame(dataforinptomodel,columns = ["inputname","inputvalue"]).transpose()
display(xforinp)
inpnonnumdata

In [ ]:
#scaling data into numbers
xforinp1 = xforinp.copy()
klist = list(xforinp1.loc["inputvalue",:])
xforinp2 =[]
for i in range(len(klist)) :
  if type(klist[i])==str or type(klist[i])==object:
    xforinp2.append(inpnonnumdata[i]) #if its string get the option selected by the user
  else:
    xforinp2.append(klist[i]) #if it not of str append the same element

In [ ]:
#unordered list of values valinp1   #the index for both unordered list depict the same value
#unordered list of columns colval1
#ordered list of columns  dfcolformodel
#ordered list of values  dfvaluesformodel
#display(list(zip(*dataforinptomodel)))

In [ ]:
##ARRANGING AS PER DATABASE
print((xforinp2))
colval1,valueinp1 = zip(*dataforinptomodel)# column name , value of the varibale
print(colval1,valueinp1,sep = "\n\n *")
dfcolformodel =[col for col in  df_all.columns if col in colval1] #getting column names as per the database
print(dfcolformodel) #dataframe column for inputing to model
dfvaluesformodel = [] #dataframe value for inputing to model
for col in dfcolformodel:
  i =colval1.index(col)  #column name
  dfvaluesformodel.append(xforinp2[i] )
print(dfvaluesformodel)

In [ ]:
#APPENDING VALUES TO THE INPUT DATABASE
xforinp1.loc["inputname",:] = dfcolformodel #test input column name for the model
xforinp1.loc['inputvalue',:] = dfvaluesformodel#test user input value for model
display(xforinp1)
xforinp3 =pd.concat([(xforinp1.loc["inputvalue",:]),xforinp1.loc["inputvalue",:]],axis =1,ignore_index = True) #getting the values user input data and doubling it by concatinating  columns based
xforinp3 =xforinp3.transpose()
xforinp3

In [ ]:

# mMs = MinMaxScaler()
# mMs.fit(xforinp3.iloc[0,:])
# xforinp3_scaled = mMs.transform(xforinp3)
# pd.DataFrame(xforinp3_scaled)

# FINAL OUTPUT

In [ ]:
modelp =modelforpred()
modelp.fit(x_scaled_resamp,y_resamp)
y_pred1 = modelp.predict(xforinp3)
y_pred1[0]

In [ ]:
riskfactor = []
for i in y_pred1:
  if i ==1:
    riskfactor.append("risky")
  else:
    riskfactor.append("not risky")  #output for the banker
riskfactor[0]

In [ ]:
riskfactor = []
for i in y_pred1:
  if i ==1:
    riskfactor.append("try again after few months, your loan is rejected")
  else:
    riskfactor.append("the loan is approved for further process")  #output for the user
riskfactor[0]

In [ ]:
time2 = time.time()
tottime = time2 - time1
tottime


In [ ]:
import colab
list(colab.__dir__())

In [ ]:
colab.__cached__

In [ ]:
colab.__builtins__